# Lifetime Value with Transaction Data

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Import Data

In [2]:
tran_df = pd.read_csv("Transactions.csv")

## Change data type

trans_date to datetime type

In [3]:
tran_df['trans_date'] = pd.to_datetime(tran_df['trans_date'])
tran_df

,customer_id,trans_date,tran_amount
0,CS5295,2013-02-11,35
1,CS4768,2015-03-15,39
2,CS2122,2013-02-26,52
3,CS1217,2011-11-16,99
4,CS1850,2013-11-20,78
...,...,...,...
124995,CS8433,2011-06-26,64
124996,CS7232,2014-08-19,38
124997,CS8731,2014-11-28,42
124998,CS8133,2013-12-14,13


## Find First Order Date

In [4]:
tran_df_groupby = tran_df.groupby(by = 'customer_id').agg({'trans_date':'min'}).reset_index()
tran_df_groupby.rename(columns={'trans_date':'first_order'}, inplace = True)
tran_df_groupby

,customer_id,first_order
0,CS1112,2011-06-15
1,CS1113,2011-05-27
2,CS1114,2011-07-14
3,CS1115,2011-08-10
4,CS1116,2011-06-27
...,...,...
6884,CS8996,2011-11-15
6885,CS8997,2011-06-08
6886,CS8998,2011-06-08
6887,CS8999,2011-07-25


## Join First Order Date to Transaction Table

In [5]:
tran_df = tran_df.merge(tran_df_groupby, on ='customer_id')
tran_df

,customer_id,trans_date,tran_amount,first_order
0,CS5295,2013-02-11,35,2011-10-27
1,CS5295,2014-11-25,54,2011-10-27
2,CS5295,2013-05-22,37,2011-10-27
3,CS5295,2012-12-24,36,2011-10-27
4,CS5295,2013-03-22,98,2011-10-27
...,...,...,...,...
124995,CS8077,2011-09-18,51,2011-08-28
124996,CS8077,2014-12-02,27,2011-08-28
124997,CS8077,2011-08-28,30,2011-08-28
124998,CS8077,2015-03-01,19,2011-08-28


## Extract Year from First Order Date

In [6]:
tran_df['first_order_year'] = tran_df['first_order'].dt.year
tran_df

,customer_id,trans_date,tran_amount,first_order,first_order_year
0,CS5295,2013-02-11,35,2011-10-27,2011
1,CS5295,2014-11-25,54,2011-10-27,2011
2,CS5295,2013-05-22,37,2011-10-27,2011
3,CS5295,2012-12-24,36,2011-10-27,2011
4,CS5295,2013-03-22,98,2011-10-27,2011
...,...,...,...,...,...
124995,CS8077,2011-09-18,51,2011-08-28,2011
124996,CS8077,2014-12-02,27,2011-08-28,2011
124997,CS8077,2011-08-28,30,2011-08-28,2011
124998,CS8077,2015-03-01,19,2011-08-28,2011


## Calculate Transaction Period

In [7]:
tran_df['trans_period'] = ((tran_df['trans_date'] - tran_df['first_order'])//np.timedelta64(1,'Y')) + 1
tran_df

,customer_id,trans_date,tran_amount,first_order,first_order_year,trans_period
0,CS5295,2013-02-11,35,2011-10-27,2011,2
1,CS5295,2014-11-25,54,2011-10-27,2011,4
2,CS5295,2013-05-22,37,2011-10-27,2011,2
3,CS5295,2012-12-24,36,2011-10-27,2011,2
4,CS5295,2013-03-22,98,2011-10-27,2011,2
...,...,...,...,...,...,...
124995,CS8077,2011-09-18,51,2011-08-28,2011,1
124996,CS8077,2014-12-02,27,2011-08-28,2011,4
124997,CS8077,2011-08-28,30,2011-08-28,2011,1
124998,CS8077,2015-03-01,19,2011-08-28,2011,4


## Find Total Amount and No. Customer for each First Order Year & Transaction Period

In [8]:
ltv_group_df =  tran_df.groupby(by=['first_order_year','trans_period'])\
                .agg({'tran_amount':'sum','customer_id':'nunique'}).reset_index()
ltv_group_df

,first_order_year,trans_period,tran_amount,customer_id
0,2011,1,2440523,6501
1,2011,2,2026788,6427
2,2011,3,2008863,6423
3,2011,4,1347974,6080
4,2012,1,114025,376
5,2012,2,90796,365
6,2012,3,82137,355
7,2012,4,7440,84
8,2013,1,3474,12
9,2013,2,1792,12


## Calculate Lifetime Value (Average Revenue)

In [9]:
ltv_group_df['LTV'] = ltv_group_df['tran_amount'] / ltv_group_df['customer_id']
ltv_group_df

,first_order_year,trans_period,tran_amount,customer_id,LTV
0,2011,1,2440523,6501,375.407322
1,2011,2,2026788,6427,315.355220
2,2011,3,2008863,6423,312.760859
3,2011,4,1347974,6080,221.706250
4,2012,1,114025,376,303.257979
5,2012,2,90796,365,248.756164
6,2012,3,82137,355,231.371831
7,2012,4,7440,84,88.571429
8,2013,1,3474,12,289.500000
9,2013,2,1792,12,149.333333


## Create Yearly Cohort

In [10]:
ltv_df = ltv_group_df[['first_order_year','trans_period','LTV']]
ltv_df = ltv_df.pivot(index = 'first_order_year', columns='trans_period', values = 'LTV')
ltv_df

trans_period,1,2,3,4
first_order_year,,,,
2011,375.407322,315.355220,312.760859,221.706250
2012,303.257979,248.756164,231.371831,88.571429
2013,289.500000,149.333333,59.000000,NaN


# for Yearly Cohort (By Date)

## Find Total Amount and No. Customer for each First Order Date & Transaction Period

In [11]:
ltv_group_date_df = tran_df.groupby(by=['first_order','trans_period'])\
                    .agg({'tran_amount':'sum','customer_id':'nunique'}).reset_index()
ltv_group_date_df

,first_order,trans_period,tran_amount,customer_id
0,2011-05-16,1,30017,85
1,2011-05-16,2,29266,85
2,2011-05-16,3,27422,84
3,2011-05-16,4,24106,84
4,2011-05-17,1,30094,77
...,...,...,...,...
1485,2013-09-04,2,22,1
1486,2013-09-13,1,401,1
1487,2013-09-13,2,138,1
1488,2013-10-26,1,166,1


## Calculate Lifetime Value (Average Revenue)

In [12]:
ltv_group_date_df['LTV'] = ltv_group_date_df['tran_amount'] / ltv_group_date_df['customer_id']
ltv_group_date_df

,first_order,trans_period,tran_amount,customer_id,LTV
0,2011-05-16,1,30017,85,353.141176
1,2011-05-16,2,29266,85,344.305882
2,2011-05-16,3,27422,84,326.452381
3,2011-05-16,4,24106,84,286.976190
4,2011-05-17,1,30094,77,390.831169
...,...,...,...,...,...
1485,2013-09-04,2,22,1,22.000000
1486,2013-09-13,1,401,1,401.000000
1487,2013-09-13,2,138,1,138.000000
1488,2013-10-26,1,166,1,166.000000


## Create Yearly Cohort

In [13]:
ltv_date_df = ltv_group_date_df[['first_order','trans_period','LTV']]
ltv_date_df = ltv_date_df.pivot(index = 'first_order', columns='trans_period', values = 'LTV')
ltv_date_df

trans_period,1,2,3,4
first_order,,,,
2011-05-16,353.141176,344.305882,326.452381,286.976190
2011-05-17,390.831169,318.116883,321.558442,304.194805
2011-05-18,386.467391,299.978261,296.266667,270.813187
2011-05-19,408.301205,319.000000,323.135802,248.626506
2011-05-20,370.406593,362.876404,339.186813,262.415730
...,...,...,...,...
2013-05-14,581.000000,168.000000,NaN,NaN
2013-07-04,386.000000,112.000000,NaN,NaN
2013-09-04,180.000000,22.000000,NaN,NaN


## Extra: Frequency, Ticket size, No.Customer cohort

In [14]:
extra_group_df = tran_df.groupby(by=['first_order_year','trans_period'])\
                    .agg({'tran_amount':'mean','customer_id':'count'}).reset_index()
freq_df = extra_group_df[['first_order_year','trans_period','customer_id']]
freq_df = freq_df.pivot(index = 'first_order_year', columns='trans_period', values = 'customer_id')
freq_df

trans_period,1,2,3,4
first_order_year,,,,
2011,37591.0,31082.0,30681.0,20514.0
2012,1953.0,1565.0,1396.0,121.0
2013,62.0,32.0,3.0,NaN


In [15]:
ts_df = extra_group_df[['first_order_year','trans_period','tran_amount']]
ts_df = ts_df.pivot(index = 'first_order_year', columns='trans_period', values = 'tran_amount')
ts_df

trans_period,1,2,3,4
first_order_year,,,,
2011,64.923067,65.207773,65.475799,65.709954
2012,58.384537,58.016613,58.837393,61.487603
2013,56.032258,56.000000,59.000000,NaN


In [16]:
cust_df = ltv_group_df[['first_order_year','trans_period','customer_id']]
cust_df = cust_df.pivot(index = 'first_order_year', columns='trans_period', values = 'customer_id')
cust_df

trans_period,1,2,3,4
first_order_year,,,,
2011,6501.0,6427.0,6423.0,6080.0
2012,376.0,365.0,355.0,84.0
2013,12.0,12.0,3.0,NaN
